In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import warnings
#Optimization
from scipy.optimize import minimize
start_date='2015-04-01'
end_date='2023-4-23'
df_nv=yf.download('NVDA',start=start_date,end=end_date)
df_ap=yf.download('AAPL',start=start_date,end=end_date)
df_ms=yf.download('MSFT',start=start_date,end=end_date)
df_goo=yf.download('GOOGL',start=start_date,end=end_date)

df_amz=yf.download('AMZN',start=start_date,end=end_date)
df_lr=yf.download('LRCX',start=start_date,end=end_date)

In [ ]:
def rolling_above(df,ma):
    # suppress all warnings
    warnings.filterwarnings("ignore")
    price=df[['Adj Close']]
    MA='MA{}'.format(str(ma))
    price[MA]=price['Adj Close'].rolling(window=int(ma)).mean()
    price=price.dropna()
    price['Position']=np.where(price[MA]<price['Adj Close'],1,-1)
    detect=price['Position'].diff()
    price['Return']=np.log(price['Adj Close']/
                           price['Adj Close'].shift(1))
    price['Strategy']=price['Return'].shift(1)*price['Position']
    price.dropna(inplace=True)
    ret=np.exp(price[['Strategy']].sum())
    warnings.filterwarnings("default")
    return ret
# define the initial guess
x0 = [200]
# define the bounds for the input variables
bound = [(180, 230)]
# integer constraint
def integer_constraint(x):
    # custom constraint function that checks if the input variable is an integer
    return np.round(x) - x
constraints = [{'type': 'ineq', 'fun': integer_constraint}]

In [ ]:
res_ms2 = minimize(lambda x: rolling_above(df_ms,x),x0, method='SLSQP',
               bounds=bound, constraints=constraints)

print("MSFT Optimized values:", res_ms2.x)
print("Function value:", res_ms2.fun)

In [ ]:
res_nv2 = minimize(lambda x: rolling_above(df_nv,x),x0, method='SLSQP',
               bounds=bound, constraints=constraints)

print("NVDA Optimized values:", res_nv2.x)
print("Function value:", res_nv2.fun)

In [ ]:
def rolling_above_strat_order_num(df,tic,ma):
    # suppress all warnings
    warnings.filterwarnings("ignore")
    price=df[['Adj Close']]
    price=price.rename(columns={'Adj Close':tic})
    MA='MA{}'.format(str(ma))
    price[MA]=price[tic].rolling(window=int(ma)).mean()
    price=price.dropna()
    price['Position']=np.where(price[MA]<price[tic],1,0)
    detect=price['Position'].diff()
    buy_indices=price.index[detect>0].to_list()
    price['Return']=np.log(price[tic]/price[tic].shift(1))
    price['Strategy']=price['Return'].shift(1)*price['Position']
    price.dropna(inplace=True)
    ret=np.exp(price[['Strategy']].sum())[0]
    no_strat=np.exp(price[['Return']].sum())[0]
    return len(buy_indices),ret,no_strat
    warnings.filterwarnings("default")
days=np.arange(150,240,10)
for df,tic in zip(dfs,tics):
    best_ct=1000
    best_ret=0
    best_day_ct=0
    best_ret_ct=0
    best_day_ret=0
    no_strat_ret=0
    print('Here we go ',tic)
    for day in days:
        cnt,ret,no_strat=rolling_above_strat_order_num(df,tic,day)
        if cnt<best_ct:
            best_ct=cnt
            best_day_ct=day
            best_ret_ct=ret
        if ret>best_ret:
            best_ret=ret
            best_day_ret=day
            no_strat_ret=no_strat
    print('Best MA to choose for ',tic,' to minimize order number is ',best_day_ct)
    print('Its order number is ',best_ct,' and its return is ',best_ret_ct)
    print('Best MA to choose for ',tic,' to maximize return is ',best_day_ret)
    print('Its return is ',best_ret,'. Comare it with ',no_strat_ret)

In [ ]:
def rolling_above_plot(df,tic,ma):
    # suppress all warnings
    warnings.filterwarnings("ignore")
    price=df[['Adj Close']]
    price=price.rename(columns={'Adj Close':tic})
    MA='MA{}'.format(str(ma))
    price[MA]=price[tic].rolling(window=int(ma)).mean()
    price=price.dropna()
    price['Position']=np.where(price[MA]<price[tic],1,0)
    detect=price['Position'].diff()
    buy_indices=price.index[detect>0].to_list()
    print('Buy at ',buy_indices)
    sell_indices=price.index[detect<0].to_list()
    print('Total number of orders made ',len(sell_indices))
    ax = price.plot(secondary_y='Position', figsize=(10, 6))
    ax.get_legend().set_bbox_to_anchor((0.25, 0.85))
    price['Return']=np.log(price[tic]/price[tic].shift(1))
    price['Strategy']=price['Return'].shift(1)*price['Position']
    price.dropna(inplace=True)
    ret=np.exp(price[['Return','Strategy']].sum())
    print(ret)
    warnings.filterwarnings("default")

In [ ]:
dfs=[df_nv,df_ap,df_ms,df_goo,df_amz,df_lr]
tics=['NVDA','AAPL','MSFT','GOOGL','AMZN','LRCX']

In [ ]:
for df,tic in zip(dfs,tics):
    print('Here we go ',tic)
    rolling_above_plot(df,tic,230)